In [4]:
# This notebook illustrates example outputs from Funman, and how to work with the ParameterSpace object it creates.

# The file scratch/hackathon/hackathon_fall_2023_demo_terarrium.py was used to generate the outputs rendered here.

SAVED_RESULTS_DIR = "./out"

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman_demo.example.pde import plot_spatial_timeseries
from IPython.display import HTML
import matplotlib.pyplot as plt
import itertools
from funman import FunmanWorkRequest, EncodingSchedule

import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
from funman import FUNMANConfig
import logging
from funman_demo import summarize_results



RESOURCES = "../resources"
# EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet", "terrarium-tests")
# MODEL_PATH = os.path.join(
#     EXAMPLE_DIR,"seird.json"
# )
# REQUEST_PATH = os.path.join(
#     EXAMPLE_DIR, "seird_request.json"
# )


EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet", "stratified")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR,"model_amr.json"
)
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "model_amr_request.json"
)


# %load_ext autoreload
# %autoreload 2

In [5]:
# Step 0: Unconstrained model, find any parameter assignment

with open(REQUEST_PATH, "r") as request:
    funman_request = FunmanWorkRequest.model_validate(json.load(request))
    funman_request.structure_parameters[0].schedules[0] = EncodingSchedule(timepoints=[0,1, 2])
    funman_request.config.substitute_subformulas = True
    funman_request.config.use_transition_symbols = True
#     funman_request.config.use_compartmental_constraints=False
#     r_H_to_D = funman_request.parameter("r_H_to_D")
#     r_H_to_D.interval.lb = 0
#     r_H_to_D.interval.ub = 1.0

    # r_E_to_I = funman_request.parameter("r_E_to_I")
    # r_E_to_I.interval.lb = 0
    # r_E_to_I.interval.ub = 1.0


    results_unconstrained_point = Runner().run(
            MODEL_PATH,
            funman_request,
            description="SEIRD demo",
            case_out_dir=SAVED_RESULTS_DIR,
    )
    # summary = summarize_results(["S", "E", "I", "R", "D"], results_unconstrained_point, ylabel="Population")
    # print(summary)

2024-02-24 14:35:06,560 - funman.server.worker - INFO - FunmanWorker running...
2024-02-24 14:35:06,566 - funman.server.worker - INFO - Starting work on: 73b12a36-418f-4419-841a-914e6e69253a
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.52000, 0.52000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
[0.00000, 0.00000) has 

In [6]:
results_unconstrained_point.parameter_space.points()[0]

{'type': 'point', 'label': 'true', 'values': {'chi_0': 1.0, 'd_0_0': 5.1706308169596694e-08, 'phi_0': 1.0, 'q': 0.52, 'd_0_1': 5.1706308169596694e-08, 'd_0_2': 5.1706308169596694e-08, 'd_0_3': 5.1706308169596694e-08, 'chi_1': 1.0, 'chi_2': 1.0, 'phi_1': 1.0, 'd_1_0': 5.1706308169596694e-08, 'd_1_1': 5.1706308169596694e-08, 'd_1_2': 5.1706308169596694e-08, 'd_1_3': 5.1706308169596694e-08, 'd_2_0': 5.1706308169596694e-08, 'd_2_1': 5.1706308169596694e-08, 'd_2_2': 5.1706308169596694e-08, 'd_2_3': 5.1706308169596694e-08, 'd_3_0': 5.1706308169596694e-08, 'd_3_1': 5.1706308169596694e-08, 'd_3_2': 5.1706308169596694e-08, 'd_3_3': 5.1706308169596694e-08, 'rho': 0.5, 'eta_0_0_0': 0.1, 'gamma': 0.2, 'eta_0_0_1': 0.1, 'eta_0_1_0': 0.1, 'eta_0_1_1': 0.1, 'eta_0_2_0': 0.1, 'eta_0_2_1': 0.1, 'eta_1_0_0': 0.1, 'eta_1_0_1': 0.1, 'eta_1_1_0': 0.1, 'eta_1_1_1': 0.1, 'eta_1_2_0': 0.1, 'eta_1_2_1': 0.1, 'eta_2_0_0': 0.1, 'eta_2_0_1': 0.1, 'eta_2_1_0': 0.1, 'eta_2_1_1': 0.1, 'eta_2_2_0': 0.1, 'eta_2_2_1': 